# Silverbox

Silverbox refers to one of the nonlinear system identification benchmarks on http://nonlinearbenchmark.org/#Silverbox. 
It is a simulation of a [Duffing oscillator](https://en.wikipedia.org/wiki/Duffing_equation), ocurring for instance in nonlinear spring pendulums.

State-space model description of the system:

$$\begin{align}
m \frac{d^2 x(t)}{dt^2} + v \frac{d x(t)}{dt} + a x(t) + b x^3(t) =&\ u(t) + w(t) \\
y(t) =&\ x(t) + e(t)
\end{align}$$

where
$$\begin{align}
m     =&\ \text{mass} \\
v     =&\ \text{viscous damping} \\
a     =&\ \text{linear stiffness} \\
b     =&\ \text{nonlinear stiffness} \\
y(t)    =&\ \text{observation (displacement)} \\
x(t)    =&\ \text{state (displacement)} \\
u(t)    =&\ \text{force} \\
e(t)    =&\ \text{measurement noise} \\
w(t)    =&\ \text{process noise}
\end{align}$$

The process noise is a Wiener process, where the increment is Gaussian distributed:

$$\begin{align}
w(t) =&\ \frac{d B(t)}{dt} \sim \mathcal{N}(0, \tau^{-1}dt)
\end{align}$$

The parameter $\tau$ represents the precision of the process. The same holds for the measurement noise.

## Solution steps

### 1. Ignore nonlinear stiffness

For now, we ignore the nonlinear stiffness component by setting the parameter $b$ to 0. The state transition thus reduces to:

$$\begin{align}
m x''(t) + v x'(t) + a x(t) = u(t) + w(t) 
\end{align}$$

### 2. Discretize

I'm using an implicit method, for stability reasons.
The backward difference for both derivative terms:

$$\begin{align}
x''(t) \approx&\ \frac{x(t) - 2x(t-h) + x(t-2h)}{h^2} = \frac{x_t - 2x_{t-1} + x_{t-2}}{(\Delta t)^2}\\
x'(t) \approx&\ \frac{x(t) - x(t-h)}{h} = \frac{x_t - x_{t-1}}{\Delta t}\\
\end{align}$$

where $\Delta t = t - (t-1) = 1$. A discretization of the Wiener process yields:

$$\begin{align}
w(t) = \frac{dB(t)}{dt} \approx \frac{B(t) - B(t-h)}{h} = \frac{B_t - B_{t-1}}{\Delta t} \sim \mathcal{N}(0, \tau^{-1}\Delta t) \, .
\end{align}$$

Let $w_t$ be a sample from $\mathcal{N}(0, \tau^{-1})$. The control signal $u(t)$ was constructed from a discrete signal, converted into an analogue signal through a zero-order-hold filter. That means it's a step-function: constant between any $t$ and $t-1$. Since it is an observed variable, we can just convert $u(t)$ straight to $u_t$. The DE can now be written as the following discrete-time system:

$$m (x_t - 2x_{t-1} + x_{t-2}) + v (x_t - x_{t-1}) + a x_t = u_t + w_t$$

Re-writing this in terms of $x_t$ yields:
$$\begin{align}
(m + v + a) x_t&\ + (-2m - v) x_{t-1} + m x_{t-2} = u_t + w_t \\
% x_t + \frac{-2m - v}{m + v + a} x_{t-1} + \frac{m}{m + v + a} x_{t-2} =&\ \frac{1}{m + v + a} u_t + \frac{1}{m + v + a} w_t \\
x_t&\ = \frac{2m + v}{m + v + a} x_{t-1} + \frac{-m}{m + v + a} x_{t-2} + \frac{1}{m + v + a} u_t + \frac{1}{m + v + a} w_t \, .
\end{align}$$


### 3. Change to shorthand notation:

I'm introducing some shorthand to clean the equation up a bit:

$$\begin{align} 
\theta_1 =&\ \frac{2m + v}{m + v + a} \\
\theta_2 =&\ \frac{-m}{m + v + a} \\
\eta =&\ \frac{1}{m + v + a} \, .
\end{align}$$

This produces:
$$\begin{align}
x_t = \theta_1 x_{t-1} + \theta_2 x_{t-2} + \eta u_t + \eta w_t
\end{align}$$

Now I'm going to absorb $\eta$ into $w_t$ (using $\mathbb{V}[aX] = a^2\mathbb{V}[X]$):

$$\begin{align}
\mathbb{V}[\eta w_t] = \eta^2 \mathbb{V}[w_t] = \eta^2 \tau^{-1}
\end{align}$$

I will rename $\eta^2 \tau^{-1}$ as $\gamma^{-1}$. This yields

$$\begin{align}
x_t = \theta_1 x_{t-1} + \theta_2 x_{t-2} + \eta u_t + \tilde{w}_t
\end{align}$$

where $\tilde{w}_t \sim \mathcal{N}(0, \gamma^{-1})$. Given four equations and four unknowns, I can recover $m$, $v$, $a$ and $\tau$ from $\theta_1$, $\theta_2$, $\eta$ and $\gamma$.

### 4. Cast to multivariate first-order form

The system now resembles an auto-regressive process:

$$ \underbrace{\begin{bmatrix} x_t \\ x_{t-1} \end{bmatrix}}_{z_t} = \underbrace{\begin{bmatrix} \theta_1 & \theta_2 \\ 1 & 0 \end{bmatrix}}_{A(\theta)} \underbrace{\begin{bmatrix} x_{t-1} \\ x_{t-2} \end{bmatrix}}_{z_{t-1}} + \underbrace{\begin{bmatrix} \eta \\ 0 \end{bmatrix}}_{B(\eta)} u_t + \begin{bmatrix} 1 \\ 0 \end{bmatrix} \tilde{w}_t \, .$$

Note that we need a two-dimensional state prior now (reminiscent of adding an initial condition on the velocity).

### 5. Convert to Gaussian probability

The state transition maps to

$$z_t \sim \mathcal{N}(A(\theta) z_{t-1} + c\eta u_t, V)$$

where $V = \begin{bmatrix} \gamma^{-1} & 0 \\ 0 & \epsilon \end{bmatrix}$ and $V^{-1} = W = \begin{bmatrix} \gamma & 0 \\ 0 & 1/\epsilon \end{bmatrix}$.

The observation likelihood maps to

$$y_t \sim \mathcal{N}(c^{\top} z_t, \sigma^2)$$

where $c = \begin{bmatrix} 1 & 0 \end{bmatrix}$ and $e_t \sim \mathcal{N}(0, \sigma^2)$.

### 6. Choose priors

I will first study a situation with known measurement noise, i.e., where $\sigma$ is fixed. The mass is a strictly positive parameter, but the damping and stiffness coefficients can be negative. As such, $m$ is modeled by a log-Normal distribution and $v$ and $a$ by a Normal. Process precision $\gamma$ is strictly positive and in this case it is more suitable to use a gamma distribution:

$$\begin{align}
\log(m) \sim&\ \mathcal{N}(m^{0}_m, v^{0}_m) \\
v \sim&\ \mathcal{N}(m^{0}_v, v^{0}_v) \\ 
a \sim&\ \mathcal{N}(m^{0}_a, v^{0}_a) \\
\gamma \sim&\ \Gamma(a^{0}_\gamma, b^{0}_\gamma) 
\end{align}$$

### Data

Let's first have a look at the data.

In [1]:
using Revise
using CSV
using DataFrames

In [2]:
using Plots
viz = false;

In [3]:
# Read data from CSV file
df = CSV.read("../data/SNLS80mV.csv", ignoreemptylines=true)
df = select(df, [:V1, :V2])

# Shorthand
input = df[:,1]
output = df[:,2]

# Time horizon
T = size(df, 1);

In [4]:
if viz
    # Plot every n-th time-point to avoid figure size exploding
    n = 10
    p1 = Plots.scatter(1:n:T, output[1:n:T], color="black", label="output", markersize=2, size=(1600,800), xlabel="time (t)", ylabel="response")
    # Plots.savefig(p1, "viz/output_signal.png")
end

In [5]:
if viz
    p2 = Plots.scatter(1:n:T, input[1:n:T], color="blue", label="output", markersize=2, size=(1600,800), xlabel="time (t)", ylabel="control")
    # Plots.savefig(p2, "viz/input_signal.png")
end

## Estimating parameters via Bayesian filtering

Implementation with ForneyLab and AR node. The AR node is locally modified from the package LAR (LAR is in dev mode).

In [6]:
using ForneyLab
using ForneyLab: unsafeMean, unsafeCov, unsafeVar, unsafePrecision
using LAR
using LAR.Node, LAR.Data
using ProgressMeter

┌ Info: Precompiling LAR [c3bc7fac-5998-4d64-8961-b7df36e0e4ce]
└ @ Base loading.jl:1260
  ** incremental compilation may be fatally broken for this module **



In [11]:
# Start graph
graph = FactorGraph()

# Static parameters (log-Normal will be exponentiated in nonlinear function)
@RV m ~ GaussianMeanPrecision(placeholder(:m_m), placeholder(:w_m)) 
@RV v ~ GaussianMeanPrecision(placeholder(:m_v), placeholder(:w_v))
@RV a ~ GaussianMeanPrecision(placeholder(:m_a), placeholder(:w_a))
@RV τ ~ Gamma(placeholder(:a_τ), placeholder(:b_τ))

# Nonlinearities
g1(m,v,a) = [(2*exp(m) + v)/(exp(m) + v + a), -exp(m)/(exp(m) + v + a)]
g2(m,v,a) = 1/(exp(m) + v + a)
g3(m,v,a,τ) = τ*(exp(m) + v + a)^2
@RV θ ~ Nonlinear{Sampling}(m, v, a, g=g1, dims=(2,))
@RV η ~ Nonlinear{Sampling}(m, v, a, g=g2, dims=(1,))
@RV γ ~ Nonlinear{Sampling}(m, v, a, τ, g=g3, dims=(1,))

# Observation selection variable
c = [1, 0]

# Measurement precision
σ = 1e4

# State prior
@RV z_t ~ GaussianMeanPrecision(placeholder(:m_z, dims=(2,)), placeholder(:w_z, dims=(2, 2)), id=:z_t)

# Autoregressive node
@RV x_t ~ AutoregressiveControl(θ, z_t, η, placeholder(:u_t), γ, id=:x_t)

# Specify likelihood
@RV y_t ~ GaussianMeanPrecision(dot(c, x_t), σ, id=:y_t)

# Placeholder for observation
placeholder(y_t, :y_t)

# Draw time-slice subgraph
ForneyLab.draw(graph)

# Infer an algorithm
q = PosteriorFactorization(z_t, x_t, θ, η, γ, m, v, a, τ, ids=[:z, :x, :θ, :η, :γ, :m, :v, :a, :τ])
algo = variationalAlgorithm(q, free_energy=false)
source_code = algorithmSourceCode(algo, free_energy=false)
eval(Meta.parse(source_code));
# println(source_code)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (20161225.0304)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 3897859526920882576 -->
 
 3897859526920882576 
 
 placeholder_b_τ 
 
<!-- 13497965966136924470 -->
 
 13497965966136924470 
 
 = 
 equ_a_1 
 
<!-- 15231577918378787231 -->
 
 15231577918378787231 
 
 𝒩 
 gaussianmeanprecision_3 
 
<!-- 13497965966136924470--15231577918378787231 -->
 
 13497965966136924470--15231577918378787231 
 
 a 
 1 out 
 1 1 
 
<!-- 1053075117831068635 -->
 
 1053075117831068635 
 
 Nonlinear{Sampling} 
 nonlinear_1 
 
<!-- 13497965966136924470--1053075117831068635 -->
 
 13497965966136924470--1053075117831068635 
 
 a 
 4 in3 
 2 2 
 
<!-- 10305124411746047069 -->
 
 10305124411746047069 
 
 placeholder_a_τ 
 
<!-- 3663401157658430378 -->
 
 3663401157658430378 
 
 clamp_1 
 
<!-- 13642108786606988292 -->
 
 13642108786606988292 
 
 placeholder_w_v 
 
<!-- 7179597653017573909 -->
 
 7179597653017573909 
 
 ARC 
 x_t 
 
<!-- 1143734541959309612 -->
 
 1143734541959309612 
 
 𝒩 
 z_t 
 
<!-- 7179597653017573909--1143734541959309612 -->
 
 7179597653017573909--1143734541959309612 
 
 z_t 
 1 out 
 2 x 
 
<!-- 3780040960177170083 -->
 
 3780040960177170083 
 
 Nonlinear{Sampling} 
 nonlinear_2 
 
<!-- 7179597653017573909--3780040960177170083 -->
 
 7179597653017573909--3780040960177170083 
 
 η 
 1 out 
 4 η 
 
<!-- 7179597653017573909--1053075117831068635 -->
 
 7179597653017573909--1053075117831068635 
 
 θ 
 1 out 
 3 θ 
 
<!-- 549406857082325448 -->
 
 549406857082325448 
 
 placeholder_u_t 
 
<!-- 7179597653017573909--549406857082325448 -->
 
 7179597653017573909--549406857082325448 
 
 u_t 
 1 out 
 5 u 
 
<!-- 9074005984413826174 -->
 
 9074005984413826174 
 
 Nonlinear{Sampling} 
 nonlinear_3 
 
<!-- 7179597653017573909--9074005984413826174 -->
 
 7179597653017573909--9074005984413826174 
 
 γ 
 1 out 
 6 γ 
 
<!-- 602892225915536807 -->
 
 602892225915536807 
 
 = 
 equ_a_2 
 
<!-- 602892225915536807--13497965966136924470 -->
 
 602892225915536807--13497965966136924470 
 
 a 
 3 3 
 2 2 
 
<!-- 602892225915536807--3780040960177170083 -->
 
 602892225915536807--3780040960177170083 
 
 a 
 4 in3 
 1 1 
 
<!-- 602892225915536807--9074005984413826174 -->
 
 602892225915536807--9074005984413826174 
 
 a 
 4 in3 
 3 3 
 
<!-- 17840753252446810218 -->
 
 17840753252446810218 
 
 placeholder_w_z 
 
<!-- 1143734541959309612--17840753252446810218 -->
 
 1143734541959309612--17840753252446810218 
 
 w_z 
 1 out 
 3 w 
 
<!-- 2821375567664623642 -->
 
 2821375567664623642 
 
 placeholder_m_z 
 
<!-- 1143734541959309612--2821375567664623642 -->
 
 1143734541959309612--2821375567664623642 
 
 m_z 
 1 out 
 2 m 
 
<!-- 3697691659022014149 -->
 
 3697691659022014149 
 
 placeholder_m_a 
 
<!-- 11790216632575188336 -->
 
 11790216632575188336 
 
 placeholder_m_m 
 
<!-- 17727907933832764883 -->
 
 17727907933832764883 
 
 𝒩 
 gaussianmeanprecision_2 
 
<!-- 17727907933832764883--13642108786606988292 -->
 
 17727907933832764883--13642108786606988292 
 
 w_v 
 1 out 
 3 w 
 
<!-- 4890055558260248677 -->
 
 4890055558260248677 
 
 placeholder_m_v 
 
<!-- 17727907933832764883--4890055558260248677 -->
 
 17727907933832764883--4890055558260248677 
 
 m_v 
 1 out 
 2 m 
 
<!-- 2277502314015428764 -->
 
 2277502314015428764 
 
 = 
 equ_v_2 
 
<!-- 2277502314015428764--3780040960177170083 -->
 
 2277502314015428764--3780040960177170083 
 
 v 
 3 in2 
 1 1 
 
<!-- 4455416755517146310 -->
 
 4455416755517146310 
 
 = 
 equ_v_1 
 
<!-- 2277502314015428764--4455416755517146310 -->
 
 2277502314015428764--4455416755517146310 
 
 v 
 3 3 
 2 2 
 
<!-- 2277502314015428764--9074005984413826174 -->
 
 2277502314015428764--9074005984413826174 
 
 v 
 3 in2 
 3 3 
 
<!-- 16885849968381140503 -->
 
 16885849968381140503 
 
 = 
 equ_m_2 
 
<!-- 16885849968381140503--378004096017717

ArgumentError: ArgumentError: The input graph contains a loop around Interface 6 (γ) of AutoregressiveControl x_t
.

FL complains about a loop in the input graph. Next attempt transforms a single vector $\phi = (m, v, a, \tau)$ into another vector $\psi = (\theta_1, \theta_2, \eta, \gamma)$.

In [16]:
# Start graph
graph = FactorGraph()

# Static parameters (log-Normal will be exponentiated in nonlinear function)
@RV ϕ ~ GaussianMeanPrecision(placeholder(:m_ϕ, dims=(4,)), placeholder(:w_ϕ, dims=(4,4)))

# Nonlinearities
g(ϕ) = [(2*exp(ϕ[1])+ϕ[2])/(exp(ϕ[1])+ϕ[2]+ϕ[3]), -exp(ϕ[1])/(exp(ϕ[1])+ϕ[2]+ϕ[3]), 1/(exp(ϕ[1])+ϕ[2]+ϕ[3]), ϕ[4]*(exp(ϕ[1])+ϕ[2]+ϕ[3])^2]
@RV ψ ~ Nonlinear{Sampling}(ϕ, g=g, dims=(4,))

# Selection variables
sel1 = [1. 0. 0. 0.; 
        0. 1. 0. 0.]
sel2 = [0. 0. 1. 0.]
sel3 = [0. 0. 0. 1.]
c = [1., 0.]

# Measurement precision
σ = 1e4

# State prior
@RV z_t ~ GaussianMeanPrecision(placeholder(:m_z, dims=(2,)), placeholder(:w_z, dims=(2, 2)), id=:z_t)

# Autoregressive node
# @RV x_t ~ AutoregressiveControl(ψ[1:2], z_t, ψ[3], placeholder(:u_t), ψ[4], id=:x_t)
@RV x_t ~ AutoregressiveControl(dot(sel1, ψ), z_t, dot(sel2, ψ), placeholder(:u_t), dot(sel3, ψ), id=:x_t)

# Specify likelihood
@RV y_t ~ GaussianMeanPrecision(dot(c, x_t), σ, id=:y_t)

# Placeholder for observation
placeholder(y_t, :y_t)

# Draw time-slice subgraph
ForneyLab.draw(graph)

# Infer an algorithm
q = PosteriorFactorization(z_t, x_t, ϕ, ψ, ids=[:z, :x, :ϕ, :ψ])
algo = variationalAlgorithm(q, free_energy=false)
source_code = algorithmSourceCode(algo, free_energy=false)
eval(Meta.parse(source_code));
# println(source_code)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (20161225.0304)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 14887015953685561082 -->
 
 14887015953685561082 
 
 placeholder_y_t 
 
<!-- 17741638425267839304 -->
 
 17741638425267839304 
 
 𝒩 
 y_t 
 
<!-- 14887015953685561082--17741638425267839304 -->
 
 14887015953685561082--17741638425267839304 
 
 y_t 
 1 out 
 1 out 
 
<!-- 1611234164323979955 -->
 
 1611234164323979955 
 
 Nonlinear{Sampling} 
 nonlinear_1 
 
<!-- 226377192558881904 -->
 
 226377192558881904 
 
 𝒩 
 gaussianmeanprecision_1 
 
<!-- 1611234164323979955--226377192558881904 -->
 
 1611234164323979955--226377192558881904 
 
 ϕ 
 1 out 
 2 in1 
 
<!-- 5900440824932702289 -->
 
 5900440824932702289 
 
 clamp_1 
 
<!-- 8096306191757840222 -->
 
 8096306191757840222 
 
 clamp_5 
 
<!-- 17741201957901401538 -->
 
 17741201957901401538 
 
 = 
 equ_ψ_1 
 
<!-- 17741201957901401538--1611234164323979955 -->
 
 17741201957901401538--1611234164323979955 
 
 ψ 
 1 out 
 1 1 
 
<!-- 6293737769280962582 -->
 
 6293737769280962582 
 
 dot 
 dotproduct_1 
 
<!-- 17741201957901401538--6293737769280962582 -->
 
 17741201957901401538--6293737769280962582 
 
 ψ 
 2 in1 
 2 2 
 
<!-- 12487786933253958372 -->
 
 12487786933253958372 
 
 dot 
 dotproduct_3 
 
<!-- 11178307880096737539 -->
 
 11178307880096737539 
 
 clamp_3 
 
<!-- 12487786933253958372--11178307880096737539 -->
 
 12487786933253958372--11178307880096737539 
 
 clamp_3 
 1 out 
 3 in2 
 
<!-- 3645308212199742118 -->
 
 3645308212199742118 
 
 clamp_4 
 
<!-- 15148599755921986584 -->
 
 15148599755921986584 
 
 placeholder_m_z 
 
<!-- 17106236979521725341 -->
 
 17106236979521725341 
 
 placeholder_m_ϕ 
 
<!-- 10131668459557795811 -->
 
 10131668459557795811 
 
 = 
 equ_ψ_2 
 
<!-- 10131668459557795811--17741201957901401538 -->
 
 10131668459557795811--17741201957901401538 
 
 ψ 
 3 3 
 2 2 
 
<!-- 10131668459557795811--12487786933253958372 -->
 
 10131668459557795811--12487786933253958372 
 
 ψ 
 2 in1 
 3 3 
 
<!-- 7417886412321976214 -->
 
 7417886412321976214 
 
 dot 
 dotproduct_2 
 
<!-- 10131668459557795811--7417886412321976214 -->
 
 10131668459557795811--7417886412321976214 
 
 ψ 
 2 in1 
 1 1 
 
<!-- 944204644299903466 -->
 
 944204644299903466 
 
 clamp_2 
 
<!-- 7417886412321976214--944204644299903466 -->
 
 7417886412321976214--944204644299903466 
 
 clamp_2 
 1 out 
 3 in2 
 
<!-- 6984833870566425214 -->
 
 6984833870566425214 
 
 𝒩 
 z_t 
 
<!-- 6984833870566425214--15148599755921986584 -->
 
 6984833870566425214--15148599755921986584 
 
 m_z 
 1 out 
 2 m 
 
<!-- 16352776986335685783 -->
 
 16352776986335685783 
 
 placeholder_w_z 
 
<!-- 6984833870566425214--16352776986335685783 -->
 
 6984833870566425214--16352776986335685783 
 
 w_z 
 1 out 
 3 w 
 
<!-- 4020872683895833073 -->
 
 4020872683895833073 
 
 dot 
 dotproduct_4 
 
<!-- 4020872683895833073--3645308212199742118 -->
 
 4020872683895833073--3645308212199742118 
 
 clamp_4 
 1 out 
 3 in2 
 
<!-- 596472060177361835 -->
 
 596472060177361835 
 
 ARC 
 x_t 
 
<!-- 4020872683895833073--596472060177361835 -->
 
 4020872683895833073--596472060177361835 
 
 x_t 
 1 y 
 2 in1 
 
<!-- 226377192558881904--17106236979521725341 -->
 
 226377192558881904--17106236979521725341 
 
 m_ϕ 
 1 out 
 2 m 
 
<!-- 14504607965934601278 -->
 
 14504607965934601278 
 
 placeholder_w_ϕ 
 
<!-- 226377192558881904--14504607965934601278 -->
 
 226377192558881904--14504607965934601278 
 
 w_ϕ 
 1 out 
 3 w 
 
<!-- 6293737769280962582--5900440824932702289 -->
 
 6293737769280962582--5900440824932702289 
 
 clamp_1 
 1 out 
 3 in2 
 
<!-- 17741638425267839304--8096306191757840222 -->
 
 17741638425267839304--8096306191757840222 
 
 clamp_5 
 1 out 
 3 w 
 
<!-- 17741638425267839304--4020872683895833073 -->
 
 17741638425267839304--4020872683895833073 
 
 variable_4 
 1 out 
 

ArgumentError: ArgumentError: The input graph contains a loop around Interface 3 (3) of Equality equ_ψ_1
.

I can't select elements of $\psi$ as input to the ARC node. I also can't use selection matrices / vectors as that produces another loop.

I could rewrite the ARC node to accept a parameter vector and send out a combined message

In [ ]:
# Looking at only the first few timepoints
# T = 100
T = size(df, 1);

# Inference parameters
num_iterations = 10

# Initialize marginal distribution and observed data dictionaries
data = Dict()
marginals = Dict()

# Initialize arrays of parameterizations
params_x = (zeros(2,T+1), repeat(.1 .*float(eye(2)), outer=(1,1,T+1)))
params_θ = (ones(2,T+1), repeat(.1 .*float(eye(2)), outer=(1,1,T+1)))
params_η = (ones(1,T+1), 0.1*ones(1,T+1))
params_γ = (0.1*ones(1,T+1), 0.01*ones(1,T+1))

# Start progress bar
p = Progress(T, 1, "At time ")

# Perform inference at each time-step
for t = 1:T

    # Update progress bar
    update!(p, t)

    # Initialize marginals
    marginals[:x_t] = ProbabilityDistribution(Multivariate, GaussianMeanPrecision, m=params_x[1][:,t], w=params_x[2][:,:,t])
    marginals[:z_t] = ProbabilityDistribution(Multivariate, GaussianMeanPrecision, m=params_x[1][:,t], w=params_x[2][:,:,t])
    marginals[:θ] = ProbabilityDistribution(Multivariate, GaussianMeanPrecision, m=params_θ[1][:,t], w=params_θ[2][:,:,t])
    marginals[:η] = ProbabilityDistribution(Univariate, GaussianMeanPrecision, m=params_η[1][1,t], w=params_η[2][1,t])
    marginals[:γ] = ProbabilityDistribution(Univariate, Gamma, a=params_γ[1][1,t], b=params_γ[2][1,t])
    
    data = Dict(:y_t => output[t],
                :u_t => input[t],
                :m_z => params_x[1][:,t],
                :w_z => params_x[2][:,:,t],
                :m_θ => params_θ[1][:,t],
                :w_θ => params_θ[2][:,:,t],
                :m_η => params_η[1][1,t],
                :w_η => params_η[2][1,t],
                :a_γ => params_γ[1][1,t],
                :b_γ => params_γ[2][1,t])

    # Iterate variational parameter updates
    for i = 1:num_iterations

        stepx!(data, marginals)
        stepθ!(data, marginals)
        stepη!(data, marginals)
        stepγ!(data, marginals)
    end

    # Store current parameterizations of marginals
    params_x[1][:,t+1] = unsafeMean(marginals[:x_t])
    params_x[2][:,:,t+1] = marginals[:x_t].params[:w]
    params_θ[1][:,t+1] = unsafeMean(marginals[:θ])
    params_θ[2][:,:,t+1] = marginals[:θ].params[:w]
    params_η[1][1,t+1] = unsafeMean(marginals[:η])
    params_η[2][1,t+1] = marginals[:η].params[:w]
    params_γ[1][1,t+1] = marginals[:γ].params[:a]
    params_γ[2][1,t+1] = marginals[:γ].params[:b]

end

### Visualize results

In [ ]:
viz = true

In [ ]:
# Extract mean of state marginals
estimated_states = params_x[1][1,2:end]

if viz
    # Plot every n-th time-point to avoid figure size exploding
    n = 10
    p1 = Plots.scatter(1:n:T, output[1:n:T], color="black", label="output", markersize=2, size=(1400,600), xlabel="time (t)", ylabel="response")
    Plots.plot!(1:n:T, estimated_states[1:n:T], color="red", linewidth=1, label="estimated")
#     Plots.savefig(p1, "viz/estimated_states01.png")
end

In [ ]:
# Extract mean of coefficient marginals
estimated_coeffs_1_mean = params_θ[1][1,2:end]
estimated_coeffs_1_std = sqrt.(inv.(params_θ[2][1,1,2:end]))
estimated_coeffs_2_mean = params_θ[1][2,2:end]
estimated_coeffs_2_std = sqrt.(inv.(params_θ[2][2,2,2:end]))

if viz
    
#     # Plot both coefficients within the same figure
#     Plots.plot(1:n:T, estimated_coeffs_1_mean[1:n:T], ribbon=[estimated_coeffs_1_std[1:n:T], estimated_coeffs_1_std[1:n:T]], color="red", label="θ_1", xlabel="time (t)", ylim=[-1.5, 1.5])
#     Plots.plot!(1:n:T, estimated_coeffs_2_mean[1:n:T], ribbon=[estimated_coeffs_2_std[1:n:T], estimated_coeffs_2_std[1:n:T]], color="blue", label="θ_2")
# #     Plots.savefig("viz/estimated_coeffs.png")
    
    # Plot both coefficients next to each other
    p2a = Plots.plot(1:n:T, estimated_coeffs_1_mean[1:n:T], ribbon=[estimated_coeffs_1_std[1:n:T], estimated_coeffs_1_std[1:n:T]], color="red", label="θ_1", xlabel="time (t)")
    p2b = Plots.plot(1:n:T, estimated_coeffs_2_mean[1:n:T], ribbon=[estimated_coeffs_2_std[1:n:T], estimated_coeffs_2_std[1:n:T]], color="blue", label="θ_2", xlabel="time (t)")
    p2 = plot(p2a, p2b, size=(1200,400))
#     Plots.savefig(p2, "viz/estimated_coeffs_subp.png")
end

In [ ]:
# Extract mean of control coefficient marginals
estimated_ccoeff_mean = params_η[1][1,2:end]
estimated_ccoeff_std = sqrt.(inv.(params_η[2][1,2:end]))

if viz
    # Plot both coefficients next to each other
    p3 = Plots.plot(1:n:T, estimated_ccoeff_mean[1:n:T], ribbon=[estimated_ccoeff_std[1:n:T], estimated_ccoeff_std[1:n:T]], color="blue", label="η", xlabel="time (t)", size=(800,600), ylim=[0.0, 0.25])
#     Plots.savefig(p3, "viz/estimated_ccoeff.png")
end

In [ ]:
# Extract mean of process precision marginals
estimated_pnoise_mean = params_γ[1][1,2:end] ./ params_γ[2][1,2:end]
estimated_pnoise_std = sqrt.(params_γ[1][1,2:end] ./ params_γ[2][1,2:end].^2)

if viz
    # Plot both coefficients next to each other
    p4 = Plots.plot(1:n:T, estimated_pnoise_mean[1:n:T], ribbon=[estimated_pnoise_std[1:n:T], estimated_pnoise_std[1:n:T]],color="blue", label="γ", xlabel="time (t)")
#     Plots.savefig(p4, "viz/estimated_pnoise.png")
end

## Solving nonlinear system of equations

We currently have estimates for $\theta_1$, $\theta_2$, and $\eta$. But we want to know the original coefficients, $m$, $v$ and $a$, which actually have a physical meaning. To obtain estimates for those, we have to solve the following nonlinear system of equations:

$$\begin{align} 
\hat{\theta}_1 =&\ \frac{2m + v}{m + v + a} \\
\hat{\theta}_2 =&\ \frac{-m}{m + v + a} \\
\hat{\eta} =&\ \frac{1}{m + v + a} \\
\hat{\gamma}^{-1} =&\ \frac{1}{\tau (m + v + a)^2}
\end{align}$$

Implementation using NLsolve.jl

In [ ]:
using NLsolve

In [ ]:
# Current estimates of parameters
global estimates = [estimated_coeffs_1_mean[end], estimated_coeffs_2_mean[end], estimated_ccoeff_mean[end], inv(estimated_pnoise_mean[end])]

In [ ]:
# Define nonlinear system of equations
function F!(F, x)
    F[1] = (2*x[1] + x[2])/(x[1] + x[2] + x[3])  - estimates[1]
    F[2] = (-x[1])/(x[1] + x[2] + x[3]) - estimates[2]
    F[3] = 1/(x[1] + x[2] + x[3]) - estimates[3]
    F[4] = 1/(x[4]*(x[1] + x[2] + x[3])^2) - estimates[4] 
end

# Jacobian of each equation
function J!(J, x)
    
    # F[1]
    J[1, 1] = (x[2] + 2*x[3])/(x[1] + x[2] + x[3])^2
    J[1, 2] = (x[3] - x[1])/(x[1] + x[2] + x[3])^2
    J[1, 3] = (-2*x[1] - x[2])/(x[1] + x[2] + x[3])^2
    J[1, 4] = 0.
    
    # F[2]
    J[2, 1] = (x[2] + x[3])/(x[1] + x[2] + x[3])^2
    J[2, 2] = x[1]/(x[1] + x[2] + x[3])^2
    J[2, 3] = x[1]/(x[1] + x[2] + x[3])^2
    J[2, 4] = 0.
    
    # F[3]
    J[3, 1] = -1/(x[1] + x[2] + x[3])^2
    J[3, 2] = -1/(x[1] + x[2] + x[3])^2
    J[3, 3] = -1/(x[1] + x[2] + x[3])^2
    J[3, 4] = 0.
    
    # F[4]
    J[4, 1] = -1/(2*x[4]*(x[1] + x[2] + x[3])^(3/2))
    J[4, 2] = -1/(2*x[4]*(x[1] + x[2] + x[3])^(3/2))
    J[4, 3] = -1/(2*x[4]*(x[1] + x[2] + x[3])^(3/2))
    J[4, 4] = -1/(x[4]^2*sqrt(x[1] + x[2] + x[3]))
    
end

# Call solver
# x_solved = nlsolve(F!, J!, [1. 1. 1. 1.])
x_solved = nlsolve(F!, [1. 1. 1. 1.], autodiff=:forward)

# Extract new estimates
global m, v, a, τ = x_solved.zero

So, in total we estimate the dynamical parameters as follows:

In [ ]:
println("m = " *string(m))
println("v = " *string(v))
println("a = " *string(a))
println("τ = " *string(τ))

Now, I'm going to estimate the dynamical parameters for the entire trajectory. Note that I could do this at inference time as well.

In [ ]:
m = zeros(T,1)
v = zeros(T,1)
a = zeros(T,1)
τ = zeros(T,1)

# Extract means of marginals
estimated_θ1 = params_θ[1][1,2:end]
estimated_θ2 = params_θ[1][2,2:end]
estimated_η = params_η[1][1,2:end]
estimated_γ = inv.(params_γ[1][1,2:end] ./ params_γ[2][1,2:end])

for t = 1:T
    
    function F!(F, x)
        F[1] = (2*x[1] + x[2])/(x[1] + x[2] + x[3])  - estimated_θ1[t]
        F[2] = (-x[1])/(x[1] + x[2] + x[3]) - estimated_θ2[t]
        F[3] = 1/(x[1] + x[2] + x[3]) - estimated_η[t]
        F[4] = 1/(x[4]*(x[1] + x[2] + x[3])^2) - estimated_γ[t] 
    end
    
    # Call solver
#     x_solved = nlsolve(F!, J!, [1. 1. 1. 1.])
    x_solved = nlsolve(F!, [0.1 0.1 0.1 0.1], autodiff=:forward)

    # Extract new estimates
    m[t], v[t], a[t], τ[t] = x_solved.zero
    
end

In [ ]:
# Plot belief evolution for mass
Plots.plot(1:n:T, m[1:n:T], color="red", label="m", xlabel="time (t)")

In [ ]:
# Plot belief evolution for friction coefficient
Plots.plot(1:n:T, v[1:n:T], color="blue", label="v", xlabel="time (t)")

In [ ]:
# Plot belief evolution for linear stiffness
Plots.plot(1:n:T, a[1:n:T], color="green", label="a", xlabel="time (t)")

In [ ]:
# Plot belief evolution for process precision
Plots.plot(1:n:T, τ[1:n:T], color="purple", label="τ", xlabel="time (t)")